In [1]:
#!gdown 1biIO64cHN5vn1BwLMYsLrUEGBaM5yYJv
#!gdown 1OLJtcoumUCo5YRmLoJxbAoJV0jQRSLTu


In [2]:
#!wget https://github.com/zcgzcgzcg1/MediaSum/blob/main/data/train_val_test_split.json

In [3]:
#!unzip news_dialogue.zip

In [4]:
#pip install tensorflow


In [5]:
!pip install wandb transformers datasets rouge-score nltk tensorflow

     |################################| 1.8 MB 21.8 MB/s            
     |################################| 325 kB 36.8 MB/s            
     |################################| 1.5 MB 29.8 MB/s            
     |################################| 170 kB 36.7 MB/s            
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |################################| 279 kB 30.7 MB/s            
     |################################| 144 kB 29.7 MB/s            
     |################################| 25.6 MB 20.8 MB/s            
     |################################| 86 kB 13.4 MB/s            
     |################################| 9.5 MB 35.8 MB/s            
     |################################| 1.1 MB 29.7 MB/s            
     |################################| 211 kB 30.9 MB/s            
     |################################| 106 kB 43.6 MB/s            
     |################################| 133 kB 30.7 MB/s            
     |###############

In [6]:
!wandb login 43dbcf5e460a2245d190448f7879f9fc34b86b4d

Traceback (most recent call last):
  File "/usr/local/bin/wandb", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/click/core.py", line 1034, in main
    _verify_python_env()
  File "/usr/local/lib/python3.6/dist-packages/click/_unicodefun.py", line 100, in _verify_python_env
    raise RuntimeError("\n\n".join(extra))
RuntimeError: Click will abort further execution because Python was configured to use ASCII as encoding for the environment. Consult https://click.palletsprojects.com/unicode-support/ for mitigation steps.

This system supports the C.UTF-8 locale which is recommended. You might be able to resolve your issue by exporting the following environment variables:

    export LC_ALL=C.UTF-8
    export LANG=C.UTF-8


In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

dataset = load_dataset('csv', data_files={'train': 'train1.csv', 'test': 'test1.csv'})

Using custom data configuration default-dd72199b151f391e


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-dd72199b151f391e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'utt', 'summary'],
        num_rows: 7500
    })
    test: Dataset({
        features: ['Unnamed: 0', 'utt', 'summary'],
        num_rows: 625
    })
})

In [9]:
prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["utt"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")

tokenized_datasets = dataset.map(preprocess_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(200))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(200))

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model = TFAutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")

#data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model,return_tensors ='tf')
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")






Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [11]:
prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["utt"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-dd72199b151f391e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-3977ee69447841dc.arrow


In [12]:
small_train_dataset

Dataset({
    features: ['Unnamed: 0', 'utt', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})

In [13]:
#pip install torch

In [14]:
batch_size = 2
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [15]:
train_dataset = small_train_dataset.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,)

validation_dataset = small_eval_dataset.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,)

generation_dataset = (
    tokenized_datasets["test"]
    .shuffle()
    .select(list(range(3)))
    .to_tf_dataset(
        batch_size=batch_size,
        columns=["input_ids", "attention_mask", "labels"],
        shuffle=False,
        collate_fn=data_collator,
    )
)

In [16]:
from transformers import AdamWeightDecay
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate = 2e-5)

# optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
import numpy as np
import nltk
from datasets import load_metric

metric = load_metric("rouge")

def metric_fn(eval_predictions):
    predictions, labels = eval_predictions
    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    for label in labels:
        label[label < 0] = tokenizer.pad_token_id  # Replace masked label tokens
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_predictions = [
        "\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_predictions
    ]
    decoded_labels = [
        "\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels
    ]
    result = metric.compute(
        predictions=decoded_predictions, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    # Add mean generated length
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)

    return result

In [19]:
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

import wandb
# Just start a W&B run, passing `sync_tensorboard=True)`, to plot your Tensorboard files
wandb.tensorboard.patch(root_logdir="./summarization_model_save/logs")
wandb.init(project='my-project')#,sync_tensorboard=True)

tensorboard_callback = TensorBoard(log_dir="./summarization_model_save/logs")

optimizer = tf.keras.optimizers.Adam()

# optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

metric_callback = KerasMetricCallback(
    metric_fn, eval_dataset=generation_dataset, predict_with_generate=True
)

callbacks = [metric_callback, tensorboard_callback,]

model.fit(
    train_dataset, validation_data=validation_dataset, epochs=2, callbacks=callbacks
)
wandb.finish()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


Epoch 1/2
100/100 [==============================] - 2223s 22s/step - loss: 9.4226 - val_loss: 8.3384
Epoch 2/2
100/100 [==============================] - 2209s 22s/step - loss: 7.3849 - val_loss: 8.9162


global_step,▁▁▅██
train/epoch_gen_len,█▁
train/epoch_loss,█▁
train/epoch_rouge1,▁█
train/epoch_rouge2,▁▁
train/epoch_rougeL,▁█
train/epoch_rougeLsum,▁█
train/global_step,▁█▁▅
validation/epoch_loss,▁█
validation/evaluation_loss_vs_iterations,▁█
validation/global_step,▁█▁
